In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "simulation"

## Prerequisites


### Example batch case

The following batch case is used as an example.


In [2]:
from models.case import BatchCase, Transaction

example_case = BatchCase(
    id="swap_usdt_usdc",
    description="Swap 100 USDT for USDC on Uniswap and transfer to Alice",
    steps=[
        Transaction(description="Approve USDT for Uniswap"),
        Transaction(description="Swap 100 USDT to USDC on Uniswap"),
        Transaction(description="Transfer USDC to Alice"),
    ],
)

### Contract addresses

Assume the following contract addresses are known.


In [3]:
# Contract addresses
USDT_ADDR = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
USDC_ADDR = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
UNISWAP_V2_ROUTER_ADDR = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

# Dummy EOA
ALICE_ADDR = "0x436f795B64E23E6cE7792af4923A68AFD3967952"
BOB_ADDR = "0x8c575b178927fF9A70804B8b4F7622F7666bB360"

### Tools

We use web3.py to create the instances of the contracts.


In [4]:
import os
import json
from web3 import Web3

INFURA_API_KEY = os.getenv("INFURA_API_KEY")
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"))

with open("abi/erc20.json") as file:
    erc20_contract_json = json.load(file)

with open("abi/uniswap_v2_router.json") as file:
    uniswap_contract_json = json.load(file)

# Contract instances
usdc_contract = web3.eth.contract(address=USDC_ADDR, abi=erc20_contract_json)
usdt_contract = web3.eth.contract(address=USDT_ADDR, abi=erc20_contract_json)
uniswap_contract = web3.eth.contract(
    address=UNISWAP_V2_ROUTER_ADDR, abi=uniswap_contract_json
)

Example usage of the contract instances.


In [5]:
def get_balance(token_address: str, address: str):
    contract_instance = web3.eth.contract(
        address=token_address, abi=erc20_contract_json
    )
    encoded = contract_instance.encode_abi("balanceOf", args=[address])
    tx = {
        "to": token_address,
        "data": encoded,
    }

    try:
        # Perform the call
        result = web3.eth.call(tx)
        # Decode the result (balance) from bytes to integer
        decoded_result = int.from_bytes(result)
        # Convert to USDC (assuming 6 decimal places)
        usdc_balance = decoded_result / 10**6
        return usdc_balance
    except Exception as e:
        # Handle any errors that occur during the call
        print(f"An error occurred: {e}")


# Get balance of Bob
print("Bob's USDT balance:", get_balance(USDT_ADDR, BOB_ADDR))
print("Bob's USDC balance:", get_balance(USDC_ADDR, BOB_ADDR))

# Get balance of Alice
print("Alice's USDT balance:", get_balance(USDT_ADDR, ALICE_ADDR))
print("Alice's USDC balance:", get_balance(USDC_ADDR, ALICE_ADDR))

Bob's USDT balance: 0.0
Bob's USDC balance: 0.0
Alice's USDT balance: 4.0
Alice's USDC balance: 4.44435


## Simulate the transaction

Refer to the [Tenderly API documentation](https://docs.tenderly.co/node/rpc-reference/ethereum-mainnet/tenderly_simulateBundle) for more details.


In [6]:
import os
import requests
import time

TENDERLY_API_KEY = os.getenv("TENDERLY_API_KEY")
url = f"https://mainnet.gateway.tenderly.co/{TENDERLY_API_KEY}"
data = {
    "id": 0,
    "jsonrpc": "2.0",
    "method": "tenderly_simulateBundle",
    "params": [
        [
            {
                "from": ALICE_ADDR,
                "to": USDC_ADDR,
                "data": usdc_contract.encode_abi(
                    "approve", args=[UNISWAP_V2_ROUTER_ADDR, 1_000_000]
                ),
                "value": "0x0",
            },
            {
                "from": ALICE_ADDR,
                "to": UNISWAP_V2_ROUTER_ADDR,
                "data": uniswap_contract.encode_abi(
                    "swapExactTokensForTokens",
                    args=[
                        1_000_000,
                        0,
                        [USDC_ADDR, USDT_ADDR],
                        ALICE_ADDR,
                        int(time.time()) + 500,
                    ],
                ),
                "value": "0x0",
            },
            {
                "from": ALICE_ADDR,
                "to": USDT_ADDR,
                "data": usdc_contract.encode_abi(
                    "transfer", args=[BOB_ADDR, 4_000_001]
                ),
                "value": "0x0",
            },
        ],
        "latest",
    ],
}
response = requests.post(url, json=data)
tx_results = response.json()["result"]
for tx in tx_results:
    status = tx.get("status", "N/A")
    asset_changes = "assetChanges" in tx
    assets = []
    if asset_changes:
        for asset in tx["assetChanges"]:
            assets.append(asset["assetInfo"]["symbol"])
    print(f"Status: {status}\n- Asset changes: {asset_changes}\n- Assets: {assets}")


def filter_json_data(json_data):
    # Fields to retain
    fields_to_keep = set(["status", "logs", "assetChanges"])

    # Iterate over each item in the list and filter the fields
    filtered_data = []
    for item in json_data:
        filtered_item = {key: item[key] for key in fields_to_keep if key in item}
        # Modify logs if they exist
        if "logs" in filtered_item:
            for log in filtered_item["logs"]:
                # Remove 'anonymous' field if it exists
                if "anonymous" in log:
                    del log["anonymous"]

                # Remove 'raw' field if it exists
                if "raw" in log:
                    del log["raw"]
        filtered_data.append(filtered_item)

    return filtered_data


tx_results = filter_json_data(tx_results)
print(tx_results)

Status: True
- Asset changes: False
- Assets: []
Status: True
- Asset changes: True
- Assets: ['usdc', 'usdt']
Status: True
- Asset changes: True
- Assets: ['usdt']
[{'status': True, 'logs': [{'name': 'Approval', 'inputs': [{'value': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'type': 'address', 'name': 'owner', 'indexed': True}, {'value': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 'type': 'address', 'name': 'spender', 'indexed': True}, {'value': '1000000', 'type': 'uint256', 'name': 'value', 'indexed': False}]}]}, {'status': True, 'assetChanges': [{'assetInfo': {'standard': 'ERC20', 'type': 'Fungible', 'contractAddress': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'symbol': 'usdc', 'name': 'USDC', 'logo': 'https://coin-images.coingecko.com/coins/images/6319/large/usdc.png?1696506694', 'decimals': 6, 'dollarValue': '0.9997310042381287'}, 'type': 'Transfer', 'from': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'to': '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f', 'rawAmount': '0xf

In [7]:
from utils.model_selector import get_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

llm = get_chat_model(temperature=0).model

system_prompt = """You will be provided with the results of a transaction simulation.
Extract the asset changes from the transaction. Do NOT modify the data."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)


class AssetChange(BaseModel):
    contract_address: str = Field(description="The contract address of the asset")
    name: str = Field(description="The name of the asset")
    symbol: str = Field(description="The symbol of the asset")
    token_decimals: int = Field(description="The decimals of the token if applicable")
    raw_amount: str = Field(description="The raw amount (hex format) of the asset")
    sender: str = Field(description="The sender of the transaction")
    receiver: str = Field(description="The receiver of the transaction")

    # @validator("contract_address")
    # def check_token_address_format(cls, v):
    #     if not v.startswith("0x"):
    #         raise ValueError("Token address must start with '0x'")
    #     if len(v) != 42:
    #         raise ValueError("Token address must be 42 characters long")
    #     return v


class SimulatedTransaction(BaseModel):
    # summary: str = Field(description="A summary of the transaction")
    is_success: bool = Field(
        description="Whether the transaction was successful. Check if the 'status' field."
    )
    asset_changes: list[AssetChange] = Field(
        description="The list of assets that changed during the transaction"
    )


chain = prompt | llm.with_structured_output(SimulatedTransaction)

result = chain.invoke({"input": tx_results})

# Todo: a tools are needed to convert the raw_amount to a readable format
for asset in result.asset_changes:
    outgoing_sign = "-" if asset.sender.lower() == ALICE_ADDR.lower() else "+"
    amount = int(asset.raw_amount, 16)
    print(f"{asset.symbol.upper()} ({asset.contract_address})")
    print(f"Change: {outgoing_sign}{amount:,} ({asset.raw_amount})")
    print("-------------")

USDC (0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48)
Change: -1,000,000 (0xf4240)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: +999,972 (0xf4224)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: -4,000,001 (0x3d0901)
-------------
